## First


In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.vgg16 import preprocess_input
from google.colab.patches import cv2_imshow
import cv2
import os


In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
rgb_model=load_model("/content/drive/MyDrive/SAR_RGB_models/RGB_model.keras")
sar_model=load_model("/content/drive/MyDrive/SAR_RGB_models/SAR_model.h5")

rgb_image_path = "/content/drive/MyDrive/SAR_RGB/RGB/test/urban/ROIs1970_fall_s2_72_p1099.png"
sar_image_path = "/content/drive/MyDrive/SAR_RGB/SAR/test/urban/ROIs1970_fall_s1_143_p1099.png"

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

def preprocess_rgb(rgb_image_path, target_size=(224, 224)):
    img = load_img(rgb_image_path, target_size=target_size)
    img_array = img_to_array(img)
    rgb_img_array = np.array([img_array]) / 255  # Create batch axis
    return rgb_img_array

def preprocess_sar(sar_image_path, target_size=(224, 224)):
    img = load_img(sar_image_path, target_size=target_size)
    img_array = img_to_array(img)
    sar_img_array = np.array([img_array]) / 255  # Create batch axis
    return sar_img_array


In [ ]:
# Function to get predictions from both models
def get_predictions(rgb_image_path, sar_image_path, rgb_model, sar_model):
    # Preprocess the images as required by your models
    rgb_image_processed = preprocess_rgb(rgb_image_path)
    sar_image_processed = preprocess_sar(sar_image_path)

    # Get predictions
    rgb_pred = rgb_model.predict(rgb_image_processed)
    sar_pred = sar_model.predict(sar_image_processed)

    return rgb_pred, sar_pred

In [ ]:
# Function to fuse predictions
def fuse_predictions(rgb_pred, sar_pred, method='average'):
    if method == 'average':
        final_pred = (rgb_pred + sar_pred) / 2
    elif method == 'max':
        final_pred = np.maximum(rgb_pred, sar_pred)
    elif method == 'weighted':
        final_pred = 0.6 * rgb_pred + 0.4 * sar_pred  # Example weights
    else:
        raise ValueError("Invalid fusion method")
    return final_pred

In [ ]:
class_names = ['agri', 'barrenland', 'grassland', 'urban']

rgb_pred, sar_pred = get_predictions(rgb_image_path, sar_image_path, rgb_model, sar_model)
final_pred = fuse_predictions(rgb_pred, sar_pred, method='max')
final_class_idx = np.argmax(final_pred, axis=1)[0]
final_class_name = class_names[final_class_idx]
# Convert to class label
final_class = np.argmax(final_pred, axis=1)
print("Final predicted class index:", final_class_idx)
print("Final predicted class name:", final_class_name)

1/1 [==============================] - 1s 571ms/step
Final predicted class index: 3
Final predicted class name: urban


## Second


In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os

In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Load the pre-trained models
rgb_model = load_model("/content/drive/MyDrive/SAR_RGB_models/RGB_model_100.keras")
sar_model = load_model("/content/drive/MyDrive/SAR_RGB_models/SAR_model.h5")


In [ ]:
# Folder path
rgb_folder_path = "/content/drive/MyDrive/SAR_RGB/RGB/test/urban"
sar_folder_path = "/content/drive/MyDrive/SAR_RGB/SAR/test/urban"

In [ ]:
# Function to preprocess images
def preprocess_image(image_path, target_size=(224, 224)):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Normalizing the image
    return img_array

In [ ]:
# Function to get predictions from both models for all images in a folder
def get_predictions_from_folder(rgb_folder_path, sar_folder_path, rgb_model, sar_model):
    rgb_images = sorted(os.listdir(rgb_folder_path))
    sar_images = sorted(os.listdir(sar_folder_path))

    all_final_preds = []

    for rgb_image_name, sar_image_name in zip(rgb_images, sar_images):
        rgb_image_path = os.path.join(rgb_folder_path, rgb_image_name)
        sar_image_path = os.path.join(sar_folder_path, sar_image_name)

        # Preprocess the images
        rgb_image_processed = preprocess_image(rgb_image_path)
        sar_image_processed = preprocess_image(sar_image_path)

        # Get predictions
        rgb_pred = rgb_model.predict(rgb_image_processed)
        sar_pred = sar_model.predict(sar_image_processed)

        # Fuse predictions (you can choose the fusion method)
        final_pred = fuse_predictions(rgb_pred, sar_pred, method='weighted')

        # Convert to class label
        final_class = np.argmax(final_pred, axis=1)
        all_final_preds.append(final_pred)

    return all_final_preds



In [ ]:
# Function to fuse predictions
def fuse_predictions(rgb_pred, sar_pred, method='weighted'):
    if method == 'average':
        final_pred = (rgb_pred + sar_pred) / 2
    elif method == 'max':
        final_pred = np.maximum(rgb_pred, sar_pred)
    elif method == 'weighted':
        final_pred = 0.2 * rgb_pred + 0.8 * sar_pred  # Example weights
    else:
        raise ValueError("Invalid fusion method")
    return final_pred



In [ ]:
# Class names
class_names = ['agri', 'barrenland', 'grassland', 'urban']

# Get predictions for all images in the specified class folder
all_predictions = get_predictions_from_folder(rgb_folder_path, sar_folder_path, rgb_model, sar_model)

# Convert predictions to class labels and print
for pred in all_predictions:
    final_class_idx = np.argmax(pred, axis=1)[0]
    final_class_name = class_names[final_class_idx]
    print("Predicted class:", final_class_name)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 633ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 583ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 656ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 577ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 994ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 729ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 656ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 567ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 659ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 576ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 640ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 567ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 629ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 549ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 640ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 543ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 645ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 562ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 622ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 934ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 787ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 569ms/ste